<a href="https://colab.research.google.com/github/yolitie/Deep-Learning/blob/main/EEG_Gabroli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numpy.random as npr
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time


#ENCODERDECODER

class EncoderDecoder(nn.Module):
  def __init__(self,encoder,decoder,src_embed, tgt_embed, generator):
    super(EncoderDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.generator = generator

  def forward(self, src, tgt, src_mask, tgt_mask):
    return self.generator(self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask))

  def encode(self,src,src_mask):
    return self.encoder(self.src_embed(src),src_mask)

  def decode(self,memory, src_mask, tgt, tgt_mask):
    return self.decoder(self.tgt_embed(tgt),memory, src_mask, tgt_mask)


#GENERATOR

class Generator(nn.Module):
  def __init__(self,d_model,output_d):
    super(Generator,self).__init__()
    self.proj = nn.Linear(d_model,output_d)

  def forward(self,x):
    return self.proj(x)


#CLONE

def clones(module,N):
  return nn.ModuleList([copy.deepcopy(module) for in range(N)])

#This function clones N identical layers.


#ENCODER

class Encoder(nn.Module):
  def __init__(self,layer,N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x,mask):
    for layer in self.layers:
      x = layer(x,mask)
    return self.norm(x)


#LAYER NORM

class LayerNorm(nn.Module):
  def __init__(self,features,eps=1e-6):
    super(LayerNorm,self):__init__()
    self.a_2= nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self,x):                 #torch.mean returns the mean value of each row of the input tensor in the given dimension dim.
    mean = x.mean(-1, keepdim = True)  #-1 refers to the last dimension, keepdim=true the output tensor is of the same size as input except in the dimensions dim where it is of size 1.
    std = x.std(-1,keepdim = True)
    return self.a_2*(x-mean)/(std+self.eps)+self.b_2



#SUBLAYER CONNECTION

class SublayerConnection(nn.Module):
  def __init__(self,size,dropout):
    super(SublayerConnection,self).__init__()
    self.norm = LayerNorm(size)      #Applies Layer Normalization over a mini-batch of inputs
    self.dropout = nn.Dropout(dropout)  #During training, randomly zeroes some of the elements of the input tensor with probability p using samples from a Bernoulli distribution. Each channel will be zeroed out independently on every forward call.

  def forward(self,x,sublayer):
    return x + self.dropout(sublayer(self.norm(x)))


#ENCODER LAYER

class EncoderLayer(nn.Module):
  def __init__(self,size,self_attn,feed_forward,dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size,dropout),2)
    self.size = size

  def forward(self,x,mask):
    x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,mask))
    




#MAKE MODEL

def make_model(src_d,tgt_d,N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
  c=copy.deepcopy  #Una copia profunda (deep copy) construye un nuevo objeto compuesto y luego, recursivamente, inserta copias en él de los objetos encontrados en el original.
  attn = MultiHeadedAttention(h,d_model) #La funcion que hemos hecho anteriormente
  ff = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout),N),    #Bloque del encoder
      Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),  #Bloque del decoder
      nn.Sequential(Embeddings(d_model,src_d),c(position)),            #Embedding del source
      nn.Sequential(Embeddings(d_model,tgt_d),c(position)),            #Embedding del target
      Generator(d_model, tgt_d))
      
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform(p)  #pytorch proporciona funciones de método de inicialización de uso común en torch.nn.init. 
  return model
